In [1]:
# HMM模型是关于时序的概率模型，描述由一个隐藏的马尔科夫链随机生成不可观测的状态随机序列，再由各个状态生成一个观测从而产生观测随机序列的过程。
# HMM的齐次马尔科夫性假设，即当前状态只和前一个状态有关，与其他时刻的状态及观测无关。
# HMM的观测独立性假设，即当前观测只和当前状态有关，与其他时刻的状态及观测无关。
# HMM模型由状态转移概率矩阵A，观测概率矩阵B，初始状态向量概率pi，描述。

# 训练数据：已经分词过的人民日报1998语料库
# 模型学习：（A, B, pi）的参数估计。训练数据已经分词完毕，使用极大似然估计。
# 模型预测：使用维特比算法。
# 模型评估：使用精确率precision, 召回率recall

In [2]:
import numpy as np
from tqdm import tqdm

# （A, B, pi）的参数估计。使用极大似然估计，其实就是算频率

In [3]:
# 在中文分词中，包含以下几种状态（词性)
# B: 词语的开头
# M: 中间词
# E: 词语的结尾
# S: 孤立的单个字

In [4]:
# 定义一个状态映射字典。方便我们定位状态在列表中对应位置
status2num = {'B': 0, 'M': 1, 'E': 2, 'S': 3}

In [5]:
# 定义状态转移概率矩阵。总共4个状态，所以4x4
A = np.zeros((4, 4))

In [6]:
# 定义观测概率矩阵
# 使用python内置的ord函数获取词的编码，编码大小为65536，总共4个状态
# 所以B矩阵4x65536
B = np.zeros((4, 65536))

In [7]:
# 初始状态，每一个句子的开头只有4种状态（词性）
pi = np.zeros(4)

In [8]:
with open('../jupyter_files/RMRB_yuliao.txt', encoding='utf-8') as file:    # 读取语料文件，需根据情况修改文件路径
    lines = file.readlines()    # 返回list

In [9]:
lines[0:5]

['迈向  充满  希望  的  新  世纪  ——  一九九八年  新年  讲话  （  附  图片  １  张  ）  \n',
 '中共中央  总书记  、  国家  主席  江  泽民  \n',
 '（  一九九七年  十二月  三十一日  ）  \n',
 '１２月  ３１日  ，  中共中央  总书记  、  国家  主席  江  泽民  发表  １９９８年  新年  讲话  《  迈向  充满  希望  的  新  世纪  》  。  （  新华社  记者  兰  红光  摄  ）  \n',
 '同胞  们  、  朋友  们  、  女士  们  、  先生  们  ：  \n']

In [10]:
# 将语料按分词切割好
lines1 = []
for line in lines:
    line = line.strip().split()
    lines1.append(line)

In [11]:
lines1[0]

['迈向',
 '充满',
 '希望',
 '的',
 '新',
 '世纪',
 '——',
 '一九九八年',
 '新年',
 '讲话',
 '（',
 '附',
 '图片',
 '１',
 '张',
 '）']

In [12]:
print((len(lines),len(lines1)))

(19056, 19056)


In [13]:
# 给分词后的词转换为BMES标记
lines2 = []
for line in lines1:
    tmp = []
    for words in line:
        if len(words) == 1:
            tmp.extend("S")
        else:
            tmp.extend("B"+"M"*(len(words)-2)+"E")
    lines2.append(tmp)

In [14]:
lines2[0]

['B',
 'E',
 'B',
 'E',
 'B',
 'E',
 'S',
 'S',
 'B',
 'E',
 'B',
 'E',
 'B',
 'M',
 'M',
 'M',
 'E',
 'B',
 'E',
 'B',
 'E',
 'S',
 'S',
 'B',
 'E',
 'S',
 'S',
 'S']

In [15]:
# 计算A矩阵
for line in tqdm(lines2):
    for i in range(len(line)-1):
        A[status2num[line[i]]][status2num[line[i+1]]] +=1

100%|█████████████████████████████████████████████████████████████████████████| 19056/19056 [00:01<00:00, 17022.65it/s]


In [16]:
A

array([[     0.,  85897., 499329.,      0.],
       [     0.,  45378.,  85897.,      0.],
       [282224.,      0.,      0., 299226.],
       [290911.,      0.,      0., 218532.]])

In [17]:
# 转换为概率
# 如果句子较长，许多个较小的概率值连乘，容易造成下溢。对于这种情况，可以使用log函数解决。
# 但是当碰到0时，log0是没有定义的，我们给每一个0的位置加上一个极小值-3.14e+100，使其有定义。
for i in range(len(A)):
    row_sum = np.sum(A[i])
    for j in range(len(A[i])):
        if A[i][j] == 0:
            A[i][j] = -3.14e+100
        else:
            A[i][j] = np.log(A[i][j]/row_sum)

In [18]:
A

array([[-3.14000000e+100, -1.91884919e+000, -1.58732900e-001,
        -3.14000000e+100],
       [-3.14000000e+100, -1.06226695e+000, -4.24145455e-001,
        -3.14000000e+100],
       [-7.22823902e-001, -3.14000000e+100, -3.14000000e+100,
        -6.64325843e-001],
       [-5.60300594e-001, -3.14000000e+100, -3.14000000e+100,
        -8.46385515e-001]])

In [19]:
# 计算pi矩阵
for line in tqdm(lines2):
    if len(line) != 0:                # 跳过没有内容的行，不然会报错
        pi[status2num[line[0]]] += 1          # 每一行的第一个字的状态为初始状态

100%|████████████████████████████████████████████████████████████████████████| 19056/19056 [00:00<00:00, 955351.98it/s]


In [20]:
pi

array([12091.,     0.,     0.,  6963.])

In [21]:
# 转换为概率
pi_sum = np.sum(pi)
for i in range(len(pi)):
    if pi[i] == 0:
        pi[i] = -3.14e+100
    else:
        pi[i] = np.log(pi[i] / pi_sum)

In [22]:
pi

array([-4.54815679e-001, -3.14000000e+100, -3.14000000e+100,
       -1.00666664e+000])

In [23]:
# 计算B矩阵 ----还是要回到lines1...
for line in tqdm(lines1):
    for words in line:
        status = []         # 记录每一分词的状态
        if len(words) == 1:
            status.extend("S")
        else:
            status.extend("B"+"M"*(len(words)-2)+"E")
        for i in range(len(words)):
            B[status2num[status[i]]][ord(words[i])] += 1


100%|██████████████████████████████████████████████████████████████████████████| 19056/19056 [00:02<00:00, 8531.09it/s]


In [24]:
# 转换为概率
for i in range(len(B)):
    row_sum = np.sum(B[i])
    for j in range(len(B[i])):
        if B[i][j] == 0:
            B[i][j] = -3.14e+100
        else:
            B[i][j] = np.log(B[i][j]/row_sum)

# hmm模型参数训练好后，使用维特比算法做分词

In [25]:
def hmm_predict(article, hmm_param):
    """
    使用hmm训练好的参数及维特比算法做分词
    param article: 待分词的文字
    param hmm_param: hmm参数，(A, B, pi)
    param return: 分词后的文字
    """
    A, B, pi = hmm_param
    article_partition = []         # 保存分词后的结果
    for line in article:
        line = line.strip()
        # 维特比算法
        # delta--长度为每一行长度，每一位有4种状态
        delta=[[0 for _ in range(4)] for _ in range(len(line))]
        # psi同理
        psi=[[0 for _ in range(4)] for _ in range(len(line))]
        psi[0][:]=[0,0,0,0]
        for i in range(4):
            delta[0][i] = pi[i]+B[i][ord(line[0])]    # 求初始时刻的delta。psi用零初始化即可，无需再求。
        for t in range(1, len(line)):
            for i in range(4):
                tmp = [delta[t-1][j]+A[j][i] for j in range(4)]        # 求t-1时刻状态转变为t时刻状态的所有可能概率取值
                delta[t][i] = max(tmp) + B[i][ord(line[t])]
                psi[t][i] = tmp.index(max(tmp))

        status = []          # 保存最优状态链
        It = delta[-1].index(max(delta[-1]))    # 已求出的最新一个时刻的状态
        status.append(It)
        for t in range(len(line)-1, 0, -1):
            status.insert(0, psi[t][It])
            It = psi[t][It]                 # 更新It
        
        # 根据状态做分词
        partition_line = ""    # 保存分词后的行结果
        for i in range(len(line)):
            partition_line += line[i]
            if (status[i]==2 or status[i]==3) and (i!=len(line)-1):                 # 如果字的状态为E或S，且不在行尾，则在末尾加空格。
                partition_line += " "
        article_partition.append(partition_line)
    return article_partition

In [26]:
with open("../jupyter_files/test.txt", encoding='utf-8') as f:
    article = f.readlines()

In [27]:
article

['当今世界正经历百年未有之大变局，我国发展的内部条件和外部环境正在发生深刻复杂变化。向外看，我们要面对世界经济深度衰退、国际贸易和投资大幅萎缩、国际金融市场动荡、国际交往受限、经济全球化遭遇逆流、一些国家保护主义和单边主义盛行、地缘政治风险上升等不利局面。\n',
 '从明年开始，我国将进入“十四五”时期，这是在全面建成小康社会基础上开启全面建设社会主义现代化国家新征程的第一个五年，意义十分重大。\n',
 '面对深刻变化的外部环境，保持战略定力，增强必胜信心，集中力量办好自己的事情，是我们应对各种风险挑战的关键。坚定不移推动经济高质量发展，以供给侧结构性改革为主线，着力加快建设实体经济、科技创新、现代金融、人力资源协同发展的产业体系，着力构建市场机制有效、微观主体有活力、宏观调控有度的经济体制，着力打造未来发展新优势，我们就一定能推动中国经济行得更稳、走得更好，在新征程上创造新的更大奇迹。']

In [28]:
partition_article = hmm_predict(article, (A, B, pi))

In [29]:
partition_article

['当今 世界 正经 历百年 未有 之大 变局 ， 我国 发展 的 内部 条件 和 外部 环境 正 在 发生 深刻 复杂 变化 。 向 外看 ， 我们 要面 对 世界 经济 深度 衰退 、 国际 贸易 和 投资 大幅 萎缩 、 国际 金融 市场 动荡 、 国际 交往 受限 、 经济 全球化 遭遇 逆流 、 一些 国家 保护 主义 和 单边 主义 盛行 、 地缘 政治 风险 上升 等 不利 局面 。',
 '从 明年 开始 ， 我国 将 进入 “ 十四 五 ” 时期 ， 这 是 在 全面 建成 小康 社会 基础 上 开启 全面 建设 社会 主义 现代化 国家 新 征程 的 第一 个 五年 ， 意义 十分 重大 。',
 '面对 深刻 变化 的 外部 环境 ， 保持 战略 定力 ， 增强 必胜 信心 ， 集中 力量 办好 自己 的 事情 ， 是 我们 应 对 各种 风险 挑战 的 关键 。 坚定 不移 推动 经济 高 质量 发展 ， 以供 给 侧结 构性 改革 为 主线 ， 着力 加快 建设 实体 经济 、 科技 创新 、 现代 金融 、 人力 资源 协同 发展 的 产业 体系 ， 着 力构 建市 场 机制 有效 、 微观 主体 有 活力 、 宏观 调控 有度 的 经济 体制 ， 着力 打造 未来 发展 新 优势 ， 我们 就 一定 能 推动 中国 经济 行得 更稳 、 走得 更 好 ， 在 新 征程 上 创造 新 的 更 大 奇迹 。']

# 评估hmm模型分词效果

In [30]:
perfect_partition_article = ['当今 世界 正 经历 百年 未有 之 大变局 ， 我国 发展 的 内部 条件 和 外部 环境 正 在 发生 深刻 复杂 变化 。 向外看 ， 我们 要 面对 世界 经济 深度 衰退 、 国际 贸易 和 投资 大幅 萎缩 、 国际 金融 市场 动荡 、 国际 交往 受限 、 经济 全球化 遭遇 逆流 、 一些 国家 保护 主义 和 单边 主义 盛行 、 地缘 政治 风险 上升 等 不利 局面 。',
 '从 明年 开始 ， 我国 将 进入 “ 十四五 ” 时期 ， 这 是 在 全面 建成 小康 社会 基础 上 开启 全面 建设 社会主义 现代化 国家 新征程 的 第一个 五年 ， 意义 十分 重大 。',
 '面对 深刻 变化 的 外部 环境 ， 保持 战略 定力 ， 增强 必胜 信心 ， 集中 力量 办好 自己 的 事情 ， 是 我们 应对 各种 风险 挑战 的 关键 。 坚定 不移 推动 经济 高质量 发展 ， 以 供给侧 结构性 改革 为 主线 ， 着力 加快 建设 实体 经济 、 科技 创新 、 现代 金融 、 人力 资源 协同 发展 的 产业 体系 ， 着力 构建 市场 机制 有效 、 微观 主体 有 活力 、 宏观 调控 有度 的 经济 体制 ， 着力 打造 未来 发展 新 优势 ， 我们 就 一定 能 推动 中国 经济 行得 更稳 、 走得 更好 ， 在 新征程 上 创造 新 的 更大 奇迹 。']

In [31]:
def to_region(segmentation):
    """
    将一行分词结果转换为区间
    param segmentation: 百年 未有 之 大变局
    return: [(0, 2), (2, 4), (4, 5), (5, 8)]
    """
    sequence = segmentation.split()
    result = []
    count0 = 0
    count1 = 0
    for words in sequence:
        count1 += len(words)
        result.append((count0, count1))
        count0 = count1
    return result

In [32]:
def hmm_performance(perfect, pred):
    """
    计算分词的精度和召回率
    param perfect: 标准分词结果
    param pred: 算法分词结果
    param return: (precision, recall)
    """
    A_size, B_size, A_cap_B_size = 0, 0, 0
    A, B = set(to_region(perfect)), set(to_region(pred))
    A_cap_B = A & B
    A_size += len(A)
    B_size += len(B)
    A_cap_B_size += len(A_cap_B)
    return A_cap_B_size / B_size * 100, A_cap_B_size / A_size * 100
    

In [33]:
# 将分词语料整合为一行
perfect_article = ""
for words in perfect_partition_article:
    perfect_article += words + " "
perfect_article = perfect_article.strip()

In [34]:
perfect_article

'当今 世界 正 经历 百年 未有 之 大变局 ， 我国 发展 的 内部 条件 和 外部 环境 正 在 发生 深刻 复杂 变化 。 向外看 ， 我们 要 面对 世界 经济 深度 衰退 、 国际 贸易 和 投资 大幅 萎缩 、 国际 金融 市场 动荡 、 国际 交往 受限 、 经济 全球化 遭遇 逆流 、 一些 国家 保护 主义 和 单边 主义 盛行 、 地缘 政治 风险 上升 等 不利 局面 。 从 明年 开始 ， 我国 将 进入 “ 十四五 ” 时期 ， 这 是 在 全面 建成 小康 社会 基础 上 开启 全面 建设 社会主义 现代化 国家 新征程 的 第一个 五年 ， 意义 十分 重大 。 面对 深刻 变化 的 外部 环境 ， 保持 战略 定力 ， 增强 必胜 信心 ， 集中 力量 办好 自己 的 事情 ， 是 我们 应对 各种 风险 挑战 的 关键 。 坚定 不移 推动 经济 高质量 发展 ， 以 供给侧 结构性 改革 为 主线 ， 着力 加快 建设 实体 经济 、 科技 创新 、 现代 金融 、 人力 资源 协同 发展 的 产业 体系 ， 着力 构建 市场 机制 有效 、 微观 主体 有 活力 、 宏观 调控 有度 的 经济 体制 ， 着力 打造 未来 发展 新 优势 ， 我们 就 一定 能 推动 中国 经济 行得 更稳 、 走得 更好 ， 在 新征程 上 创造 新 的 更大 奇迹 。'

In [35]:
pred_article = ""
for words in partition_article:
    pred_article += words + " "
pred_article = pred_article.strip()

In [36]:
pred_article

'当今 世界 正经 历百年 未有 之大 变局 ， 我国 发展 的 内部 条件 和 外部 环境 正 在 发生 深刻 复杂 变化 。 向 外看 ， 我们 要面 对 世界 经济 深度 衰退 、 国际 贸易 和 投资 大幅 萎缩 、 国际 金融 市场 动荡 、 国际 交往 受限 、 经济 全球化 遭遇 逆流 、 一些 国家 保护 主义 和 单边 主义 盛行 、 地缘 政治 风险 上升 等 不利 局面 。 从 明年 开始 ， 我国 将 进入 “ 十四 五 ” 时期 ， 这 是 在 全面 建成 小康 社会 基础 上 开启 全面 建设 社会 主义 现代化 国家 新 征程 的 第一 个 五年 ， 意义 十分 重大 。 面对 深刻 变化 的 外部 环境 ， 保持 战略 定力 ， 增强 必胜 信心 ， 集中 力量 办好 自己 的 事情 ， 是 我们 应 对 各种 风险 挑战 的 关键 。 坚定 不移 推动 经济 高 质量 发展 ， 以供 给 侧结 构性 改革 为 主线 ， 着力 加快 建设 实体 经济 、 科技 创新 、 现代 金融 、 人力 资源 协同 发展 的 产业 体系 ， 着 力构 建市 场 机制 有效 、 微观 主体 有 活力 、 宏观 调控 有度 的 经济 体制 ， 着力 打造 未来 发展 新 优势 ， 我们 就 一定 能 推动 中国 经济 行得 更稳 、 走得 更 好 ， 在 新 征程 上 创造 新 的 更 大 奇迹 。'

In [37]:
hmm_precision, hmm_recall = hmm_performance(perfect_article, pred_article)

In [38]:
print("hmm precision: %f" % hmm_precision)
print("hmm recall: %f" % hmm_recall)

hmm precision: 85.281385
hmm recall: 89.545455
